In [1]:
import grammars as gm

Define grammar for test and it's transition/emission probabilities:

In [10]:
grammar3 = [('E', 'AE'), ('E', 'BE'),
            ('A', 'BA'), ('A', 'BE'),
            ('B', 'AB'), ('B', 'AE'),

            ('E', 'e'), ('E', 'a'), ('E', 'b'), ('E', 'c'),
            ('A', 'e'), ('A', 'a'), ('A', 'b'), ('A', 'c'),
            ('B', 'e'), ('B', 'a'), ('B', 'b'), ('B', 'c')]

ts3 = {('E', 'AE'): 0.5, ('E', 'BE'): 0.5,
       ('A', 'BA'): 0.5, ('A', 'BE'): 0.5,
       ('B', 'AB'): 0.5, ('B', 'AE'): 0.5}

es3 = {('E', 'e'): 0.91, ('E', 'a'): 0.03,
       ('E', 'b'): 0.03, ('E', 'c'): 0.03,

       ('A', 'a'): 0.7, ('A', 'b'): 0.2,
       ('A', 'c'): 0.05, ('A', 'e'): 0.05,

       ('B', 'a'): 0.2, ('B', 'b'): 0.7,
       ('B', 'c'): 0.05, ('B', 'e'): 0.05}


Inside algorithm (see. Durbin p. 254):<br><br>

init:<br>
$\alpha(:, i, i)=e_{v}(x_{i})$<br>
main:<br>
$\alpha(v, i, j)=\sum_{y=1}^{M}\sum_{z=1}^{M}\sum_{k=i}^{j-1}\alpha(y, i, k)\alpha(z, k+1, j)t_{v}(y,z)$<br>
termination:<br>
$P(x|\theta)=\alpha(1,L,1)$


In [11]:
ires = gm.inside(sent=list('aabcbe'), grammar=grammar3,
                 ts=ts3, es=es3, debug=False)
p, alpha = ires

In [12]:
print("alpha:")
print(alpha)

alpha:
[[[ 0.03        0.0135      0.0083775   0.00765529  0.0009397   0.02458389]
  [ 0.          0.03        0.0135      0.0102525   0.00109785  0.02762369]
  [ 0.          0.          0.03        0.0135      0.001215    0.02862064]
  [ 0.          0.          0.          0.03        0.0015      0.031395  ]
  [ 0.          0.          0.          0.          0.03        0.4095    ]
  [ 0.          0.          0.          0.          0.          0.91      ]]

 [[ 0.7         0.073       0.0129075   0.00896442  0.00187439  0.01746655]
  [ 0.          0.7         0.023       0.01437     0.00236461  0.0213415 ]
  [ 0.          0.          0.2         0.028       0.0034575   0.02941298]
  [ 0.          0.          0.          0.05        0.00575     0.0273975 ]
  [ 0.          0.          0.          0.          0.2         0.336     ]
  [ 0.          0.          0.          0.          0.          0.05      ]]

 [[ 0.2         0.0805      0.120795    0.00784517  0.00783118  0.02167635]
 

In [13]:
print("p(sent|grammar) = alpha(1, L, 1):")
print(p)

p(sent|grammar) = alpha(1, L, 1):
0.0245838891309


Outside algorithm (see. Durbin p. 255):<br><br>
init:<br>
$\beta("E", 1, L)=1$<br>
$\beta(:, 1, L)=0$<br>
main:<br>
$\beta(v, i, j)=\sum_{y,z}\sum_{k=1}^{i-1}\alpha(z, k, i-1)\beta(y, k, j)t_{y}(z,v)
     +\sum_{y,z}\sum_{k=j+1}^{L}\alpha(z, j+1, k)\beta(y, i, k)t_{y}(v,z)$<br>
termination:<br>
$P(x|\theta)=\sum_{v=1}^{M}\beta(v,i,i)e_{v}(x_{i})$

In [15]:
ores = gm.outside(alpha, sent=list("aabcbe"), grammar=grammar3,
                  ts=ts3, es=es3,
                  debug=False)
pb, beta = ores

In [16]:
print("beta:")
print(beta)

beta:
[[[ 0.          0.          0.          0.          0.          1.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]]

 [[ 0.02616165  0.0318703   0.03047363  0.370825    0.455       0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.          0.        ]]

 [[ 0.03135368  0.0265724   0.03217988  0.257075    0.455       0.        ]
  

In [17]:
print("p(sent|grammar) = \sum_{v \in [1 .. M]} beta(i, i, v) * e_{v}(x_{i}):")
print(pb)

p(sent|grammar) = \sum_{v \in [1 .. M]} beta(i, i, v) * e_{v}(x_{i}):
[0.024583889130937498, 0.0, 0.0, 0.0, 0.0, 0.0]


EM algorithm (see. Durbin p. 255):<br><br>
$t_{v}(y, z) = \frac{c(v->yz)}{c(v)}$<br>
$e_{v}(a) = \frac{c(v->a)}{c(v)}$


In [18]:
t1, e1 = gm.em_step(alpha, beta, sent=list("aabcbe"),
                    grammar=grammar3,
                    ts=ts3, es=es3)

In [19]:
print("estimated transitions:")
print(t1)


estimated transitions:
{('E', 'AE'): 0.55336546681217558, ('E', 'BE'): 0.44663453318782442, ('A', 'BA'): 0.21034596855890389, ('A', 'BE'): 0.063232885679005438, ('B', 'AB'): 0.54709545140057314, ('B', 'AE'): 0.10213270369104736}


In [20]:
print("estimated emission:")
print(e1)


estimated emission:
{('E', 'e'): 0.0, ('E', 'a'): 0.0, ('E', 'b'): 0.0, ('E', 'c'): 0.0, ('A', 'a'): 0.7264211457620906, ('A', 'b'): 0.0, ('A', 'c'): 0.0, ('A', 'e'): 0.0, ('B', 'a'): 0.35077184490837959, ('B', 'b'): 0.0, ('B', 'c'): 0.0, ('B', 'e'): 0.0}


All rules like $W_{v}->W_{y}W_{z}$ and $W_{v}-> a$ sum to 1 for same $W_{v}$:

In [21]:
for w in ["E", "A", "B"]:
    print(sum([t1[(v, yz)] for (v, yz) in t1 if v == w]
               + [e1[(v, e)] for (v, e) in e1 if v == w]))

1.0
1.0
1.0
